# Homework 2: Word Similarity

## General info

<b>Due date</b>: 1pm, Sunday April 1st

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be quantifying the similarity between pairs of words using the structure of WordNet and word co-occurrence in the Brown corpus, using PMI, LSA, and word2vec. You will quantify how well these methods work by comparing to a carefully filtered human annotated gold-standard.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> It is recommended to use Python 2 but we accept Python 3 solutions</b>. Make sure you state which version you used in the beggining of this notebook.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called Similarity-353. You need to first obtain this data set, which can be downloaded <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip">here</a>. The file we will be using is called *combined.tab*. Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words.

Assume the file *combined.tab* is located <b>in the same folder as this notebook</b>. You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of dictionaries). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this assignment, we will be treating the <i>paragraphs</i> of the Brown corpus as our "documents", you can iterate over them by using the `paras` method of the corpus reader. You should start by creating a Python list where each element of the list is a set containing the word <b>types</b> from a different paragraph of the Brown corpus: the words should be lower-cased and lemmatized before they are added to the set (keep it around, because you will need this list again later on). Then, using the information in this corpus, calculate document frequencies and remove from your test set any word pairs where at least one of the two words has a document frequency of less than 10 in this corpus. 

The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either having only one sense (i.e. only one synset), or where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least five and at least five times larger than the next most common sense. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove any word pairs from the test set where at least one of the words does not contain a single primary sense or if the single primary sense is not a noun.

When you have applied these two filtering steps, print out all the pairs in your filtered test set (if you have done this correctly, the total should be more than 10, but less than 50).

(1.5 marks)

In [1]:
tab_file = open("combined.tab")

# stripped out the header
tab_file.readline()

dictionary = {}

for line in tab_file:
    line = line.strip('\n')
    items = line.split('\t')
    key = (items[0], items[1])
    value = float(items[2])
    dictionary[key] = value

print(len(dictionary))

352


In [2]:
import nltk

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

In [3]:
from nltk.corpus import brown
import re

paras_lst = []

# get paragraphs of the Brown corpus, storing in a list of sets
for paragraph in brown.paras():
    word_set = set()
    for sent in paragraph:
        for word in sent:
            lemma = lemmatizer.lemmatize(word.lower())
            word_set.add(lemma)
    
    paras_lst.append(word_set)

In [4]:
# first filtering

# get the document frequency of a given word
def get_doc_frequency(word, lst):
    count = 0
    for doc in lst:
        if word in doc:
            count += 1
            
    return count

filtered_dictionary = dict()

for k, v in dictionary.items():
    if get_doc_frequency(k[0],paras_lst) >= 10 and get_doc_frequency(k[1],paras_lst) >= 10:
        filtered_dictionary[k] = v
        
print(len(filtered_dictionary))

222


In [5]:
# second filtering

from nltk.corpus import wordnet as wn

word_primary_sense = {}

def is_not_ambiguous(word):
    if len(wn.synsets(word)) <= 1 and wn.synsets(word)[0].pos:
        word_primary_sense[word] = wn.synsets(word)[0]
        return True
    
    counts = []
    
    # get counts of lemmas associating with each synset:
    for synset in wn.synsets(word):
        count = 0
        for lemma in synset.lemmas():
            if lemma.name() == word:
                count += lemma.count()
                
        counts.append(count)
        
    max_count = max(counts)
    primary_sense = wn.synsets(word)[counts.index(max_count)]
    
    counts.sort(reverse=True)

    if counts[0] >= 5 and counts[0] >= 5 * counts[1] and primary_sense.pos() == 'n':
        word_primary_sense[word] = primary_sense
        return True
    
    return False

final_dict = {}

for (k, v) in filtered_dictionary.items():
    if is_not_ambiguous(k[0]) and is_not_ambiguous(k[1]):
        final_dict[k] = v

final_dict

{('baby', 'mother'): 7.85,
 ('brother', 'monk'): 6.27,
 ('canyon', 'landscape'): 7.53,
 ('car', 'automobile'): 8.94,
 ('century', 'year'): 7.59,
 ('coast', 'forest'): 3.15,
 ('coast', 'hill'): 4.38,
 ('coast', 'shore'): 9.1,
 ('computer', 'laboratory'): 6.78,
 ('doctor', 'personnel'): 5.0,
 ('equipment', 'maker'): 5.91,
 ('hotel', 'reservation'): 8.03,
 ('journey', 'car'): 5.85,
 ('journey', 'voyage'): 9.29,
 ('luxury', 'car'): 6.47,
 ('monk', 'slave'): 0.92,
 ('planet', 'people'): 5.75,
 ('professor', 'doctor'): 6.62,
 ('psychology', 'doctor'): 6.42,
 ('psychology', 'health'): 7.23,
 ('psychology', 'mind'): 7.69,
 ('psychology', 'science'): 6.71,
 ('school', 'center'): 3.44,
 ('soap', 'opera'): 7.94,
 ('stock', 'egg'): 1.81,
 ('type', 'kind'): 8.97,
 ('word', 'similarity'): 4.75}

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Wu-Palmer scores derived from the hypernym relationships in WordNet, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. When you're done,  print out the Python dictionary of word pair/similarity mappings. 

(0.5 marks)

In [6]:
wup_dict = {}

for k in final_dict.keys():
    value = wn.wup_similarity(word_primary_sense[k[0]], word_primary_sense[k[1]])
    wup_dict[k] = value

wup_dict

{('baby', 'mother'): 0.5,
 ('brother', 'monk'): 0.5714285714285714,
 ('canyon', 'landscape'): 0.3333333333333333,
 ('car', 'automobile'): 1.0,
 ('century', 'year'): 0.8333333333333334,
 ('coast', 'forest'): 0.16666666666666666,
 ('coast', 'hill'): 0.6666666666666666,
 ('coast', 'shore'): 0.9090909090909091,
 ('computer', 'laboratory'): 0.35294117647058826,
 ('doctor', 'personnel'): 0.13333333333333333,
 ('equipment', 'maker'): 0.5,
 ('hotel', 'reservation'): 0.375,
 ('journey', 'car'): 0.09523809523809523,
 ('journey', 'voyage'): 0.8571428571428571,
 ('luxury', 'car'): 0.1111111111111111,
 ('monk', 'slave'): 0.6666666666666666,
 ('planet', 'people'): 0.18181818181818182,
 ('professor', 'doctor'): 0.5,
 ('psychology', 'doctor'): 0.1111111111111111,
 ('psychology', 'health'): 0.21052631578947367,
 ('psychology', 'mind'): 0.5714285714285714,
 ('psychology', 'science'): 0.9411764705882353,
 ('school', 'center'): 0.13333333333333333,
 ('soap', 'opera'): 0.2222222222222222,
 ('stock', 'egg')

**Instructions:** Next, you will calculate Positive PMI (PPMI) for your word pairs using statistics derived from the Brown: you should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PPMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, appearing in the same paragraph), but you should only calculate PPMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. When you have calculated PMI for all the pairs, your code should print out the Python dictionary of word-pair/PPMI-similarity mappings.

(1 mark)

In [7]:
import math

def get_PPMI(word1,word2):
    word1_count = 0
    word2_count = 0
    both_count = 0
    total_count = 0.0
    for para in paras_lst:
        total_count += 1
        if word1 in para:
            word1_count += 1
            if word2 in para:
                both_count += 1
                    
        if word2 in para:
            word2_count += 1
    
    if both_count == 0:
        return 0
    return max(math.log((both_count/total_count)/((word1_count/total_count)*(word2_count/total_count)), 2), 0)

In [8]:
ppmi_dict = {}

for k in final_dict.keys():
    value = get_PPMI(k[0], k[1])
    ppmi_dict[k] = value

ppmi_dict

{('baby', 'mother'): 3.1068514542000756,
 ('brother', 'monk'): 2.8992677183777067,
 ('canyon', 'landscape'): 0,
 ('car', 'automobile'): 3.284928059255019,
 ('century', 'year'): 0.85521193298008,
 ('coast', 'forest'): 3.0505076829814297,
 ('coast', 'hill'): 1.2130606957673897,
 ('coast', 'shore'): 4.630747773460183,
 ('computer', 'laboratory'): 0,
 ('doctor', 'personnel'): 2.2186218696012423,
 ('equipment', 'maker'): 4.283313403192924,
 ('hotel', 'reservation'): 2.891047211572738,
 ('journey', 'car'): 0,
 ('journey', 'voyage'): 0,
 ('luxury', 'car'): 2.272328022475385,
 ('monk', 'slave'): 0,
 ('planet', 'people'): 0.4092477799069862,
 ('professor', 'doctor'): 0,
 ('psychology', 'doctor'): 3.5625762708186035,
 ('psychology', 'health'): 0,
 ('psychology', 'mind'): 2.7796743924855387,
 ('psychology', 'science'): 5.078497127110109,
 ('school', 'center'): 0.744045575429721,
 ('soap', 'opera'): 4.221195813265069,
 ('stock', 'egg'): 1.8174736272140593,
 ('type', 'kind'): 0.6500752376975433,
 (

**Instructions:** Next, you will derive similarity scores using the LSA method, i.e. apply SVD and truncate to get a dense vector and then use cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use truncatedSVD in Sci-kit learn to produce dense vectors of length 500, and then use cosine similarity to produce similarities for your word pairs. Print out the corresponding Python dictionary.

(1 mark)

In [9]:
brown_matrix = [dict.fromkeys(para, 1) for para in paras_lst]

from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
brown_matrix = vectorizer.fit_transform(brown_matrix)

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500, random_state=1)
brown_matrix = svd.fit_transform(brown_matrix.transpose())

print(brown_matrix.shape)

(44539, 500)


In [10]:
from scipy.spatial.distance import cosine as cos_distance

LSA_dict = {}

for k in final_dict.keys():
    vector_1 = (vectorizer.transform([{k[0]:1}]))
    vector_2 = (vectorizer.transform([{k[1]:1}]))
    value = cos_distance(vector_1 * brown_matrix,
                         vector_2 * brown_matrix)
    LSA_dict[k] = 1 - value

LSA_dict

{('baby', 'mother'): 0.31262707457518002,
 ('brother', 'monk'): 0.095607193126501677,
 ('canyon', 'landscape'): 0.11045437823174487,
 ('car', 'automobile'): 0.36835123080281096,
 ('century', 'year'): 0.069121801694166929,
 ('coast', 'forest'): 0.13257538966701687,
 ('coast', 'hill'): 0.20413351365787014,
 ('coast', 'shore'): 0.37604123033558379,
 ('computer', 'laboratory'): 0.10853471326789588,
 ('doctor', 'personnel'): 0.087124415298997726,
 ('equipment', 'maker'): 0.24218922584864855,
 ('hotel', 'reservation'): 0.1019967308704155,
 ('journey', 'car'): 0.00093779186665288172,
 ('journey', 'voyage'): 0.1067097698558348,
 ('luxury', 'car'): 0.099888725176074389,
 ('monk', 'slave'): -0.051476571749058397,
 ('planet', 'people'): 0.031670991954421912,
 ('professor', 'doctor'): 0.091992048712529417,
 ('psychology', 'doctor'): 0.1331089264412032,
 ('psychology', 'health'): 0.039882939001020468,
 ('psychology', 'mind'): 0.10958356344732245,
 ('psychology', 'science'): 0.3099539403509467,
 ('s

**Instructions:** Next, you will derive a similarity score from word2vec vectors, using the Gensim interface. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes), but that's okay, you won't be marked based on the speed of this. You should extract the similarites you need directly from the Gensim model, put them in a Python dictionary, and print them out.

(0.5 mark)

In [11]:
import gensim

model = gensim.models.Word2Vec(brown.sents(), size=500, iter=50)

w2v_dict = {}

for k in final_dict.keys():
    value = model.wv.similarity(k[0], k[1])
    w2v_dict[k] = value

w2v_dict

C:\Users\39809\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


{('baby', 'mother'): 0.22425195790590827,
 ('brother', 'monk'): 0.026679671837613834,
 ('canyon', 'landscape'): 0.15767883095051602,
 ('car', 'automobile'): 0.16765047322103682,
 ('century', 'year'): 0.30145498340963506,
 ('coast', 'forest'): 0.27828324688462058,
 ('coast', 'hill'): 0.43762964246436586,
 ('coast', 'shore'): 0.40730656359950118,
 ('computer', 'laboratory'): 0.16542038744510845,
 ('doctor', 'personnel'): -0.054791447393612885,
 ('equipment', 'maker'): 0.20342307162865983,
 ('hotel', 'reservation'): 0.052791506144157591,
 ('journey', 'car'): 0.19353588068752339,
 ('journey', 'voyage'): 0.46703766825064591,
 ('luxury', 'car'): 0.1314503817477794,
 ('monk', 'slave'): 0.0068935948629975825,
 ('planet', 'people'): -0.00039956828335393265,
 ('professor', 'doctor'): 0.10510651633600979,
 ('psychology', 'doctor'): 0.0021570987403512383,
 ('psychology', 'health'): 0.17303005954798401,
 ('psychology', 'mind'): 0.056262348689774158,
 ('psychology', 'science'): 0.32910080237149569,



**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries, and print out the result for each (be sure to label them!). Hint: All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(0.5 mark)


In [12]:
from scipy.stats import pearsonr 

def get_pearsonr(dict1, dict2):
    similarity_1 = []
    similarity_2 = []
    for key1, value1 in dict1.items():
        for key2, value2 in dict2.items():
            if key1 == key2:
                similarity_1.append(value1)
                similarity_2.append(value2)
    return pearsonr(similarity_1, similarity_2)

print("Wu-Palmer result:", get_pearsonr(final_dict, wup_dict))
print("PPMI result:", get_pearsonr(final_dict, ppmi_dict))
print("LSA result:", get_pearsonr(final_dict, LSA_dict))
print("word2vec result:", get_pearsonr(final_dict, w2v_dict))

Wu-Palmer result: (0.45669274063664006, 0.016636665403273413)
PPMI result: (0.21406331437316534, 0.28364702836785155)
LSA result: (0.35761141064042334, 0.067044558679085486)
word2vec result: (0.32490927695152805, 0.098207780220966795)


## Bonus: Improving the correlation

**Instructions:** To get the extra credit, you should try to get a noticeably (at least 0.05) higher correlation than that of the highest correlation that you found in the main assignment. To accomplish this, you can use any resources included or accessible via NLTK EXCEPT the Google word2vec vectors. Half of your mark here will be based on the thought and effort involved in your solution (if you tried something sensible that didn't work, show us), the other half will be based on the actual performance increase, which you should demonstrate. For full credit, be sure to explain exactly what you are doing.

(1 bonus mark) 

In [13]:
# try to combine two similarity by averaging them
def get_ensemble_values(dict1, dict2):
    
    ensemble_dict = {}
    
    factor1=1.0/sum(dict1.values())
    factor2=1.0/sum(dict2.values())
    
    for key1, value1 in dict1.items():
        for key2, value2 in dict2.items():
            if key1 == key2:
                ensemble_dict[key1] = (value1 * factor1 + value2 * factor2) / 2

    return ensemble_dict


ensemble_dict = get_ensemble_values(LSA_dict, wup_dict)
print("final result:", get_pearsonr(final_dict, ensemble_dict))

final result: (0.4634327851736203, 0.014908590661004627)


## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.